In [1]:
import pandas as pd
import scanpy as sc
import numpy as np
import anndata as an

# 2015 Data

In [2]:
data_path = f"/nfs/turbo/umms-indikar/shared/projects/cell_cycle/data/RNA_pipeline_ouputs/countMatrix/counts.raw.txt"

""" Load the raw expression """
df_2015 = pd.read_csv(data_path, index_col=0).T
print(f"{df_2015.shape=}")
df_2015.head()

df_2015.shape=(18, 19393)


geneName,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
S1a,12,0,0,0,0,150,0,330,473,0,...,299,710,50,164,287,283,257,636,913,450
S1b,5,0,0,0,0,133,0,342,464,0,...,309,705,48,176,312,293,315,675,946,470
S2a,5,0,0,0,0,122,0,507,565,0,...,747,2497,33,125,286,432,383,921,1175,674
S2b,9,0,2,0,0,137,0,563,519,0,...,838,2632,31,157,361,458,353,922,1242,696
S3a,2,0,0,0,0,143,0,535,481,0,...,747,1767,48,152,338,588,432,779,1074,743


In [3]:
data_path = f"/nfs/turbo/umms-indikar/shared/projects/myod/data/rnaseq/2018_rna/countMatrix/counts.raw.txt"

""" Load the raw expression """
df_2018 = pd.read_csv(data_path, index_col=0).T
print(f"{df_2018.shape=}")
df_2018.head()

df_2018.shape=(48, 19393)


geneName,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
63246_T0R1,12,1,3595,0,1,462,19,324,304,1,...,292,279,50,171,239,10,638,3152,757,781
63252_T1R1,26,1,5795,1,0,640,13,501,1162,0,...,704,600,80,206,450,14,1625,4735,1401,1296
63249_T2R1,6,0,3903,0,0,504,4,281,471,0,...,280,262,96,203,315,9,811,1812,853,627
63261_T3R1,14,0,4714,1,3,217,20,448,597,0,...,1382,1655,90,271,417,51,1994,3375,1156,1613
63258_T4R1,13,0,2423,0,0,265,10,205,463,0,...,610,639,29,113,216,11,789,3183,660,618


In [4]:
# merge em
df = pd.concat([df_2018, df_2015])
print(f"{df.shape=}")
df = df.reset_index()
df = df.rename(columns={'index' : 'data_id'})
df = df.set_index('data_id')
df.head()

df.shape=(66, 19393)


geneName,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
data_id,,,,,,,,,,,,,,,,,,,,,
63246_T0R1,12,1,3595,0,1,462,19,324,304,1,...,292,279,50,171,239,10,638,3152,757,781
63252_T1R1,26,1,5795,1,0,640,13,501,1162,0,...,704,600,80,206,450,14,1625,4735,1401,1296
63249_T2R1,6,0,3903,0,0,504,4,281,471,0,...,280,262,96,203,315,9,811,1812,853,627
63261_T3R1,14,0,4714,1,3,217,20,448,597,0,...,1382,1655,90,271,417,51,1994,3375,1156,1613
63258_T4R1,13,0,2423,0,0,265,10,205,463,0,...,610,639,29,113,216,11,789,3183,660,618


# Build anndata

In [5]:
# 2015 metadata
tmap = {
    'S1a' : 0,
    'S1b' : 0,
    'S2a' : 0,
    'S2b' : 0,
    'S3a' : 1,
    'S3b' : 1,
    'S4a' : 2,
    'S4b' : 2,
    'S5a' : 3,
    'S5b' : 3,
    'S6a' : 4,
    'S6b' : 4,
    'S7a' : 5,
    'S7b' : 5,
    'S8a' : 6,
    'S8b' : 6,
    'S9a' : 7,
    'S9b' : 7,
}

# 2018 metadata
meta_path = "/nfs/turbo/umms-indikar/shared/projects/myod/data/rnaseq_metadata.csv"
meta_2018 = pd.read_csv(meta_path)
meta_2018['sample_id'] = meta_2018['sample_id'].astype(int)
meta_2018 = meta_2018[['sample_id', 'timepoint', 'hour']]
meta_2018['sample_id'] = meta_2018['sample_id'].astype(str)
meta_2018.head()

,sample_id,timepoint,hour
0,63246,1,-48
1,63252,2,0
2,63249,3,8
3,63261,1,16
4,63258,2,24


In [6]:
obs_names = df.index.to_list()
var_names = df.columns.to_list()

"""BUILD OBSERVATION DATAFRAME"""
obs = pd.DataFrame({'data_id' : df.index})
obs['dataset'] = np.where(obs['data_id'].str.contains("R"), "liu_2018", "chen_2015")
obs['sample_id'] = obs['data_id'].apply(lambda x: x.split("_")[0])

# merge 2018 metadata 
obs = pd.merge(obs, meta_2018, 
               how='left', 
               left_on='sample_id',
               right_on='sample_id',)

# adjust 2015 columns
obs['timepoint'] = np.where(obs['dataset'] == 'chen_2015', obs['data_id'].map(tmap), obs['timepoint'])
obs['hour'] = np.where(obs['dataset'] == 'chen_2015', obs['timepoint'] * 8, obs['hour'])

# add n_counts
obs['n_counts'] = df.sum(axis=1).values

# flag the control timepoints
control_points = [
    'S1a',  
    'S1b',
    "63246_T0R1",
    "63247_T0R2",
    "63248_T0R3",
]

obs['control'] = obs['data_id'].isin(control_points)
obs = obs.set_index('data_id')


"""BUILD THE VAR DATAFRAME"""
gene_path = "/nfs/turbo/umms-indikar/shared/projects/geneformer/token_mappings.csv"
genes = pd.read_csv(gene_path)
genes = genes[genes['gene_name'].isin(var_names)]
genes = genes.drop_duplicates(subset='gene_name')

var = pd.DataFrame({'gene_name' : var_names})

var = pd.merge(var, genes, how='left',
               left_on='gene_name',
               right_on='gene_name',
              )

var = var.set_index('gene_name')
var['ensembl_id'] = var['gene_id']

"""BUILD ADATA"""
adata = an.AnnData(df.to_numpy())
adata.obs_names = obs_names
adata.var_names = var_names

adata.obs = obs
adata.var = var

out_path = "/nfs/turbo/umms-indikar/shared/projects/geneformer/data/rajapakse_lab_data.h5ad"
adata.write(out_path)
adata

AnnData object with n_obs × n_vars = 66 × 19393
    obs: 'dataset', 'sample_id', 'timepoint', 'hour', 'n_counts', 'control'
    var: 'gene_id', 'token_id', 'Chromosome', 'Source', 'Feature', 'Start', 'End', 'Score', 'Strand', 'Frame', 'gene_version', 'gene_source', 'gene_biotype', 'transcript_id', 'transcript_version', 'transcript_name', 'transcript_source', 'transcript_biotype', 'tag', 'ccds_id', 'exon_number', 'exon_id', 'exon_version', 'protein_id', 'protein_version', 'transcript_support_level', 'ensembl_id'

In [7]:
adata.obs.head()

,dataset,sample_id,timepoint,hour,n_counts,control
data_id,,,,,,
63246_T0R1,liu_2018,63246,1.0,-48.0,11940999,True
63252_T1R1,liu_2018,63252,2.0,0.0,18063509,False
63249_T2R1,liu_2018,63249,3.0,8.0,11031474,False
63261_T3R1,liu_2018,63261,1.0,16.0,16761043,False
63258_T4R1,liu_2018,63258,2.0,24.0,8244802,False


In [8]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
df.sum(axis=1)

In [ ]:
adata.var.head()

In [ ]:
obs[obs['dataset'] == 'chen_2015']

In [ ]:
# [x for x in obs['data_id'].to_list() if "63248" in x]